In [ ]:
import cv2
import numpy as np

# Function to generate and save an 8x8 checkerboard pattern
def save_checkerboard_image(filename, square_size_cm):
    rows, cols = 8, 8
    pixels_per_cm = 100  # Adjust this based on your printing resolution

    square_size_px = int(square_size_cm * pixels_per_cm)
    pattern = np.zeros((rows * square_size_px, cols * square_size_px), dtype=np.uint8) # 8 squrares x 200

    for i in range(0, rows, 2): #from 0 to 6
        for j in range(0, cols, 2):
            pattern[i * square_size_px: (i + 1) * square_size_px, j * square_size_px: (j + 1) * square_size_px] = 255

    for i in range(1, rows, 2): # from 1 to 7
        for j in range(1, cols, 2):
            pattern[i * square_size_px: (i + 1) * square_size_px, j * square_size_px: (j + 1) * square_size_px] = 255

    cv2.imwrite(filename, pattern)
    print(f"Checkerboard pattern saved to: {filename}")

# Save the checkerboard pattern as an image file with 2 cm squares
save_checkerboard_image("checkerboard_pattern_8x8.png", square_size_cm=2)


In [1]:
for i in range(0, 8, 2):
    print(i)

0
2
4
6


In [23]:
import numpy as np
import cv2

pattern = np.zeros((8 * 200, 8 * 200), dtype=np.uint8) # 8 squrares x 200
for i in range(0, 4, 2): #from 0 to 6
        for j in range(0, 4, 2):
            pattern[i * 200: (i + 1) * 200, j * 200: (j + 1) * 200] = 255
            cv2.imshow('Checkerboard Pattern', pattern)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


In [2]:
import cv2 as cv
import os

CHESS_BOARD_DIM = (7, 7)

n = 0  # image_counter

# checking if  images dir is exist not, if not then create images directory
image_dir_path = "images"

CHECK_DIR = os.path.isdir(image_dir_path)
# if directory does not exist create
if not CHECK_DIR:
    os.makedirs(image_dir_path)
    print(f'"{image_dir_path}" Directory is created')
else:
    print(f'"{image_dir_path}" Directory already Exists.')

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


def detect_checker_board(image, grayImage, criteria, boardDimension):
    ret, corners = cv.findChessboardCorners(grayImage, boardDimension)
    if ret == True:
        corners1 = cv.cornerSubPix(grayImage, corners, (3, 3), (-1, -1), criteria)
        image = cv.drawChessboardCorners(image, boardDimension, corners1, ret)

    return image, ret


cap = cv.VideoCapture(0)

while True:
    _, frame = cap.read()
    copyFrame = frame.copy()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    image, board_detected = detect_checker_board(frame, gray, criteria, CHESS_BOARD_DIM)
    # print(ret)
    cv.putText(frame,f"saved_img : {n}",(30, 40),cv.FONT_HERSHEY_PLAIN,1.4,(0, 255, 0),2,cv.LINE_AA,)

    cv.imshow("frame", frame)
    cv.imshow("copyFrame", copyFrame)

    key = cv.waitKey(1)

    if key == ord("q"):
        break
    if key == ord("s") and board_detected == True:
        # storing the checker board image
        cv.imwrite(f"{image_dir_path}/image{n}.png", copyFrame)

        print(f"saved image number {n}")
        n += 1  # incrementing the image counter
cap.release()
cv.destroyAllWindows()

print("Total saved Images:", n)

"imageess" Directory is created
saved image number 0
saved image number 1
saved image number 2
saved image number 3
saved image number 4
saved image number 5
saved image number 6
saved image number 7
Total saved Images: 8


In [5]:
import cv2 as cv
import os
import numpy as np

# Checker board size
CHESS_BOARD_DIM = (7, 7)

# The size of Square in the checker board.
SQUARE_SIZE = 20  # millimeters

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


calib_data_path = "calib_data"
CHECK_DIR = os.path.isdir(calib_data_path)


if not CHECK_DIR:
    os.makedirs(calib_data_path)
    print(f'"{calib_data_path}" Directory is created')

else:
    print(f'"{calib_data_path}" Directory already Exists.')

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
obj_3D = np.zeros((CHESS_BOARD_DIM[0] * CHESS_BOARD_DIM[1], 3), np.float32)

obj_3D[:, :2] = np.mgrid[0 : CHESS_BOARD_DIM[0], 0 : CHESS_BOARD_DIM[1]].T.reshape(-1, 2)
obj_3D *= SQUARE_SIZE
print(obj_3D)

# Arrays to store object points and image points from all the images.
obj_points_3D = []  # 3d point in real world space
img_points_2D = []  # 2d points in image plane.

# The images directory path
image_dir_path = "imageess"

files = os.listdir(image_dir_path)
for file in files:
    print(file)
    imagePath = os.path.join(image_dir_path, file)
    # print(imagePath)

    image = cv.imread(imagePath)
    grayScale = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(image, CHESS_BOARD_DIM, None)
    if ret == True:
        obj_points_3D.append(obj_3D)
        corners2 = cv.cornerSubPix(grayScale, corners, (3, 3), (-1, -1), criteria)
        img_points_2D.append(corners2)

        img = cv.drawChessboardCorners(image, CHESS_BOARD_DIM, corners2, ret)

cv.destroyAllWindows()
# h, w = image.shape[:2]
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    obj_points_3D, img_points_2D, grayScale.shape[::-1], None, None)
print("calibrated")

print("duming the data into one files using numpy ")
np.savez(
    f"{calib_data_path}/MultiMatrix",
    camMatrix=mtx,
    distCoef=dist,
    rVector=rvecs,
    tVector=tvecs,
)


"calib_data" Directory already Exists.
[[  0.   0.   0.]
 [ 20.   0.   0.]
 [ 40.   0.   0.]
 [ 60.   0.   0.]
 [ 80.   0.   0.]
 [100.   0.   0.]
 [120.   0.   0.]
 [  0.  20.   0.]
 [ 20.  20.   0.]
 [ 40.  20.   0.]
 [ 60.  20.   0.]
 [ 80.  20.   0.]
 [100.  20.   0.]
 [120.  20.   0.]
 [  0.  40.   0.]
 [ 20.  40.   0.]
 [ 40.  40.   0.]
 [ 60.  40.   0.]
 [ 80.  40.   0.]
 [100.  40.   0.]
 [120.  40.   0.]
 [  0.  60.   0.]
 [ 20.  60.   0.]
 [ 40.  60.   0.]
 [ 60.  60.   0.]
 [ 80.  60.   0.]
 [100.  60.   0.]
 [120.  60.   0.]
 [  0.  80.   0.]
 [ 20.  80.   0.]
 [ 40.  80.   0.]
 [ 60.  80.   0.]
 [ 80.  80.   0.]
 [100.  80.   0.]
 [120.  80.   0.]
 [  0. 100.   0.]
 [ 20. 100.   0.]
 [ 40. 100.   0.]
 [ 60. 100.   0.]
 [ 80. 100.   0.]
 [100. 100.   0.]
 [120. 100.   0.]
 [  0. 120.   0.]
 [ 20. 120.   0.]
 [ 40. 120.   0.]
 [ 60. 120.   0.]
 [ 80. 120.   0.]
 [100. 120.   0.]
 [120. 120.   0.]]
image0.png
image1.png
image2.png
image3.png
image4.png
image5.png
image6.png
i

In [10]:
print("loading data stored using numpy savez function\n \n \n")

data = np.load(f"{calib_data_path}/MultiMatrix.npz")

camMatrix = data["camMatrix"]
distCof = data["distCoef"]
rVector = data["rVector"]
tVector = data["tVector"]

print("loaded calibration data successfully")

loading data stored using numpy savez function
 
 

loaded calibration data successfully


In [11]:
camMatrix

array([[746.01053083,   0.        , 303.74604778],
       [  0.        , 566.69443733, 271.87291385],
       [  0.        ,   0.        ,   1.        ]])

In [8]:
distCof

array([[ 0.13821325, -0.78655081,  0.03428718,  0.01617438, -2.30099568]])

In [9]:
rVector

array([[[-0.38563012],
        [-0.19928219],
        [ 0.1507342 ]],

       [[ 0.17394473],
        [ 0.87245276],
        [ 1.33787587]],

       [[-0.03493166],
        [-0.28121377],
        [ 1.45447057]],

       [[ 0.06588886],
        [ 0.09792278],
        [ 0.02363187]],

       [[ 0.75809488],
        [ 0.02486846],
        [ 0.08954453]],

       [[ 0.21730784],
        [-0.63985962],
        [ 0.07184427]],

       [[-0.47244118],
        [ 0.45756452],
        [-1.47086064]],

       [[ 0.0968573 ],
        [-0.11649727],
        [ 1.52257915]]])

In [7]:
camMatrix

array([[746.01053083,   0.        , 303.74604778],
       [  0.        , 566.69443733, 271.87291385],
       [  0.        ,   0.        ,   1.        ]])

In [4]:
import numpy as np

objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
objp *= 20
objp

array([[  0.,   0.,   0.],
       [ 20.,   0.,   0.],
       [ 40.,   0.,   0.],
       [ 60.,   0.,   0.],
       [ 80.,   0.,   0.],
       [100.,   0.,   0.],
       [120.,   0.,   0.],
       [  0.,  20.,   0.],
       [ 20.,  20.,   0.],
       [ 40.,  20.,   0.],
       [ 60.,  20.,   0.],
       [ 80.,  20.,   0.],
       [100.,  20.,   0.],
       [120.,  20.,   0.],
       [  0.,  40.,   0.],
       [ 20.,  40.,   0.],
       [ 40.,  40.,   0.],
       [ 60.,  40.,   0.],
       [ 80.,  40.,   0.],
       [100.,  40.,   0.],
       [120.,  40.,   0.],
       [  0.,  60.,   0.],
       [ 20.,  60.,   0.],
       [ 40.,  60.,   0.],
       [ 60.,  60.,   0.],
       [ 80.,  60.,   0.],
       [100.,  60.,   0.],
       [120.,  60.,   0.],
       [  0.,  80.,   0.],
       [ 20.,  80.,   0.],
       [ 40.,  80.,   0.],
       [ 60.,  80.,   0.],
       [ 80.,  80.,   0.],
       [100.,  80.,   0.],
       [120.,  80.,   0.],
       [  0., 100.,   0.],
       [ 20., 100.,   0.],
 

In [5]:
objp[:,:2]

array([[0., 0.],
       [1., 0.],
       [2., 0.],
       [3., 0.],
       [4., 0.],
       [5., 0.],
       [6., 0.],
       [0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [0., 2.],
       [1., 2.],
       [2., 2.],
       [3., 2.],
       [4., 2.],
       [5., 2.],
       [6., 2.],
       [0., 3.],
       [1., 3.],
       [2., 3.],
       [3., 3.],
       [4., 3.],
       [5., 3.],
       [6., 3.],
       [0., 4.],
       [1., 4.],
       [2., 4.],
       [3., 4.],
       [4., 4.],
       [5., 4.],
       [6., 4.],
       [0., 5.],
       [1., 5.],
       [2., 5.],
       [3., 5.],
       [4., 5.],
       [5., 5.],
       [6., 5.]], dtype=float32)